In [7]:
import warnings
import logging


warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [8]:
from utils import make_environments
from utils import pearl_utils
from configs  import defaults

from utils.reward_functions import log_reward_function,cumulative_reward_function,sharpe_reward_function
from utils. utils import make_hidden_dims
import optuna
from optuna.samplers import TPESampler
from neuralforecast.core import NeuralForecast
from Pearl.pearl.utils.instantiations.environments.gym_environment import GymEnvironment
from Pearl.pearl.utils.functional_utils.train_and_eval.online_learning import online_learning
import datetime

import numpy as np
import pickle
import boto3
import Keys

In [9]:
s3=boto3.client('s3', aws_access_key_id=Keys.AWS_ACCESS_KEY, aws_secret_access_key=Keys.AWS_SECRET_KEY)
agent_path=f'Agent/pearl_{defaults.model_name}_model.pkl'

s3.download_file('coinbasetradehistory',f'pearl_{defaults.model_name}_model.pkl',agent_path)


In [10]:
from configs import defaults

In [11]:
# model=NeuralForecast.load('MultiHeadForecastingModel/')

In [12]:
reward_functions=[log_reward_function,cumulative_reward_function,sharpe_reward_function]
train_env,test_env=make_environments.make_envs(reward_function=log_reward_function)
study_name=f"{defaults.model_name}"
storage_name="sqlite:///PearlHPTuning.sqlite3"

test_env.observation_space.shape,train_env.action_space.n


Seed set to 4
Seed set to 3


['data/binanceus-DOGEUSDT-1h.pkl']


100%|██████████| 78/78 [00:00<00:00, 11607.03it/s]
1it [00:00, 17.72it/s]
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Predicting: |          | 0/? [00:00<?, ?it/s]

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Predicting: |          | 0/? [00:00<?, ?it/s]

2024-10-22 16:00:00 2024-11-21 16:00:00


((24,), np.int64(2))

In [13]:
today=datetime.datetime.now().strftime('%Y-%m-%d')
study = optuna.load_study(study_name=study_name,
                            storage=storage_name,

                            )


In [14]:
# print(f"Best value: {study.best_value} (params: {study.best_params})")
best_trials=study.best_trials
best_trials

[FrozenTrial(number=9, state=TrialState.COMPLETE, values=[1396.0087663206205, 11.52], datetime_start=datetime.datetime(2024, 11, 21, 12, 17, 17, 341102), datetime_complete=datetime.datetime(2024, 11, 21, 12, 25, 42, 454267), params={'reward_function': 1, 'algorithm': 'dqn', 'n_layers': 3, 'n_units': 64, 'training_rounds': 9, 'learning_rate': 9.279788909089143e-05, 'discount_factor': 0.9878636535584397, 'batch_size': 128, 'target_update_freq': 10, 'soft_update_tau': 0.13835597466298716, 'is_conservative': True, 'lstm': True, 'conservative_alpha': 0.6219557714034156, 'learn_after_episode': False, 'learning_steps': 62, 'n_epochs': 100}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'reward_function': CategoricalDistribution(choices=(0, 1, 2)), 'algorithm': CategoricalDistribution(choices=('dqn', 'ddqn')), 'n_layers': IntDistribution(high=3, log=False, low=1, step=1), 'n_units': CategoricalDistribution(choices=(64, 128, 256, 512)), 'training_rounds': IntDistributio

In [15]:


agent_path=f'Agent/pearl_{defaults.model_name}_model.pkl'

agent_path


'Agent/pearl_DOGEUSDTSPOT_model.pkl'

In [16]:
[i for i in range(5)]

[0, 1, 2, 3, 4]

In [ ]:
best_agents={}
for i in range(4):
    agent,learning_params=pearl_utils.load_agent_from_study(study_path=storage_name,
                                            study_name=study_name,
                                            action_space_dim=train_env.action_space.n,
                                            observation_space_dim=train_env.observation_space.shape[0],
                                            version=i)
    
    agent,profit,n_trades=pearl_utils.train_production_agent(agent,
                                learning_params,
                                train_env=train_env,
                                test_env=test_env,
                                save_path=agent_path)
    best_agents[profit]=agent
agent=best_agents[max(best_agents)]
    

[I 2024-11-21 15:55:17,087] Using an existing study with name 'DOGEUSDTSPOT' instead of creating a new one.
  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/16800 [00:00<?, ?it/s]

  0%|          | 0/100 [03:22<?, ?it/s]


Testing Return AVG Profit: 1284.0353666124383, AVG Number of Trades: 6.16


  0%|          | 0/100 [00:00<?, ?it/s]

...

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/500 [23:41<?, ?it/s]


Testing Return AVG Profit: 1009.2835954782475, AVG Number of Trades: 21.04


  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

In [ ]:
# agent,learning_params=pearl_utils.load_agent_from_study(study_path="sqlite:///pearl_hyper_parameters.sqlite3",
#                                             study_name='pearl-2024-11-12-hp-search',
#                                             action_space_dim=2,
#                                             observation_space_dim=30,
#                                             version=1)

# agent=pearl_utils.train_production_agent(agent,
#                              learning_params,
#                              train_env=train_env,
#                              test_env=train_env,
#                              save_path=agent_path)

In [ ]:
best_agents,max(best_agents)

NameError: name 'best_agents' is not defined

In [ ]:
profit,n_trades=pearl_utils.test_pearl_model(agent,test_env)
print(f"Testing Return AVG Profit: {profit}, AVG Number of Trades: {n_trades}")

Testing Return AVG Profit: 1012.7186048904254, AVG Number of Trades: 71.72


In [ ]:
# pickle.dump(agent,open(agent_path,'wb'))



In [ ]:

import Keys

s3=boto3.client('s3', aws_access_key_id=Keys.AWS_ACCESS_KEY, aws_secret_access_key=Keys.AWS_SECRET_KEY)
s3.upload_file(agent_path,'coinbasetradehistory',agent_path.split('/')[-1],)


In [ ]:
# pickle.dump(agent.policy_learner.state_dict(),open(agent_path,'wb'))

In [ ]:
# weights=pickle.load(open(agent_path,'rb'))

In [ ]:
# agent.policy_learner.load_state_dict(weights)

In [ ]:

# agent,learning_params=pearl_utils.load_agent_from_study(study_path="sqlite:///pearl_hyper_parameters.sqlite3",
#                                         study_name='pearl-2024-11-12-hp-search',
#                                         action_space_dim=2,
#                                         observation_space_dim=30)
# agent=load_agent_weights(agent,weight_path=agent_path)

In [ ]:
profit,n_trades=pearl_utils.test_pearl_model(agent,test_env)
print(f"Testing Return AVG Profit: {profit}, AVG Number of Trades: {n_trades}")

Testing Return AVG Profit: 1004.5338109306196, AVG Number of Trades: 75.17
